# CREATION DE LA DAFRAME FINALE support a notre M.L.

Fusion de fichiers csv 1 par table

## Attention pour fonctionner 2 fichiers sont indispensables :
- ../DATA/list_codecommune_leg2022_50inscrits.txt <br/>List de toutes les clefs necessaires
- ../DATA/EXPORT_CSV/final_TAR_YYY_leg2022com <br/>Fichier target des resultats aux legislatives 2022 qui initialise notre dataframe

### PROCEDURE de traitement de dataframe
### 1 Initialisation avec taget
### 2 Parcours des fichiers csv du repertoire
    TESTS / actions
    - A: commence par final
    - B: validation nb de lignes 
    - C: validation des entetes
    - D: validation des clefs (depuis list_codecommune_leg2022_50inscrits.txt)
    - E: recuperation de la liste des bons prefixes a traiter (file_name.split('_')[1:-1])
    - F: recuperation de la dataframe avec les colonnes qui disposent des prefixes souhaités
    - G: fusion (merge) sur la colonne codecommune
    
### 3 [NON TRAITE] : remplacement des Nan avec voisin le plus proche

### 4 [NON TRAITE] : Export CSV 

    

In [1]:
import warnings
warnings.filterwarnings("ignore")

import os
import pandas as pd
import numpy as np

## LISTER LES FICHIERS A TRAITER

In [2]:
# INPUT : Repertoire à parcourir
# OUTPUT : Dico [nom du fichier]: prefix des colonnes à traiter

def trouver_fichiers_csv(directory):
    fichiers_csv = {}
    files = os.listdir(directory)
    for file in files:
        # file_path = os.path.join(directory, file)
        file_name, file_extension = os.path.splitext(file)
        if file_extension == '.csv' :
            infos = file_name.split('_')
            if infos[0] =='final' and len(infos[1])==3 :
                fichiers_csv[file] = infos[1:-1]
    return fichiers_csv



In [3]:
path='../DATA/EXPORT_CSV/'
list_csv = trouver_fichiers_csv(path)
list_csv 

{'final_RSA_rsacommunes.csv': ['RSA'],
 'final_AGE_agesexcommunes.csv': ['AGE'],
 'final_BFL_basesfiscalescommunes.csv': ['BFL'],
 'final_PIB_pibcommunes-ratio-et-tot.csv': ['PIB'],
 'final_GEO_correspondance-code-insee-code-postal.csv': ['GEO'],
 'final_KP0_capitalimmobilier.csv': ['KP0'],
 'final_REV_revcommunes-revmoyfoy.csv': ['REV'],
 'final_KP1_capitalimmobiliercommunes.csv': ['KP1'],
 'final_PO0_popcommuneselecteurs.csv': ['PO0'],
 'final_PRO_proprietairescommunes.csv': ['PRO'],
 'final_EMP_empfoncommunes.csv': ['EMP'],
 'final_DIP_diplomescommunes.csv': ['DIP'],
 'final_CED_crimesdelits.csv': ['CED'],
 'final_CSP_cspcommunes.csv': ['CSP'],
 'final_TAR_YYY_leg2022comm.csv': ['TAR', 'YYY'],
 'final_TER_terrescommunes.csv': ['TER'],
 'final_NAT_naticommunes.csv': ['NAT']}

## FIXER LES INDEXES

In [4]:
# Specify the file path
file_path = '../DATA/list_codecommune_leg2022_50inscrits.txt'

# Open the text file and read lines into a list
with open(file_path, 'r', encoding='utf-8') as file:
    lines = [line.strip() for line in file.readlines()]

# Print or manipulate the list of lines as needed
print(lines[0:10], len(lines))

['01001', '01002', '01004', '01005', '01006', '01007', '01008', '01009', '01010', '01011'] 33830


# IMPORTS

In [5]:
def load_verif_df (file, path, listinfo, lines, cols_entete = ['codecommune'] ) :
    file_path = path + file
    print(f"\nTRAITEMENT : {file_path}")
    if os.path.exists(file_path):
        # IMPORT CSV
        df_from_csv = pd.read_csv(file_path, encoding='utf-8', sep=';', low_memory=False)
        print(f"   df_from_csv : {df_from_csv.shape}")
        # TEST DIM
        if len(lines) == len(df_from_csv) :
            # VALIDATION ENTETES
            cols = df_from_csv.columns
 
            # VALIDATION DES CLEFS
            if set(lines) == set(df_from_csv['codecommune'].tolist()):                                            
                # LIST DES COLONNES A RECUP & FILTRE
                cols_select = cols_entete + [col for col in cols[4:] if any(col.startswith(prefix) for prefix in listinfo)]
                print(f"   > {len(cols_select)-1} COLONNES A TRAITER  {cols_select}")
                return df_from_csv[cols_select]
            else : 
                print(f"LISTE DES CLEFS NON RESPECTEE")

        else : 
            print(f"ERREUR NB. LIGNES : {len(lines)} != {len(df_from_csv)}")
    else:
        print(f"FICHIER {file_path} NON TROUVE")
    return pd.DataFrame()

### Initialisation

In [6]:
synthese = {}

### IMPOTER LA TARGET

(dico = TAR)

In [7]:
for dico_fil in list_csv : 
    if 'TAR' in list_csv[dico_fil] :
        df_final = load_verif_df (dico_fil, path, list_csv[dico_fil], lines, cols_entete = ['dep', 'nomdep', 'codecommune', 'nomcommune'])
        synthese[dico_fil] = df_final.shape[1]
        print (df_final.shape)



TRAITEMENT : ../DATA/EXPORT_CSV/final_TAR_YYY_leg2022comm.csv
   df_from_csv : (33830, 22)
   > 14 COLONNES A TRAITER  ['dep', 'nomdep', 'codecommune', 'nomcommune', 'TAR_inscrits', 'TAR_votants', 'TAR_exprimes', 'YYY_pvoixNUP', 'YYY_pvoixECO', 'YYY_pvoixENS', 'TAR_electeurs', 'YYY_pvoixLR_UDI', 'YYY_pvoixEXTREMD', 'YYY_pvoixCENTRE', 'YYY_pvoixOTR']
(33830, 15)


### FUSIONNER LES AUTRES FICHIERS DU DICO 
(hors TAR)

In [8]:
print(f"DIMENSIONS df_final EN ENTREE :  {df_final.shape}")
for dico_fil in list_csv : 
    if 'TAR' not in list_csv[dico_fil] :
        df = load_verif_df (dico_fil, path, list_csv[dico_fil], lines)
        synthese[dico_fil] = df.shape[1]-1
        df_final = pd.merge(df_final, df, on='codecommune', how='inner')
        print(f"DIMENSIONS df_final EN SORTIE :  {df_final.shape}")


DIMENSIONS df_final EN ENTREE :  (33830, 15)

TRAITEMENT : ../DATA/EXPORT_CSV/final_RSA_rsacommunes.csv
   df_from_csv : (33830, 9)
   > 4 COLONNES A TRAITER  ['codecommune', 'RSA_nrsa2017', 'RSA_nrsa2021', 'RSA_prsa2017', 'RSA_prsa2021']
DIMENSIONS df_final EN SORTIE :  (33830, 19)

TRAITEMENT : ../DATA/EXPORT_CSV/final_AGE_agesexcommunes.csv
   df_from_csv : (33830, 89)
   > 84 COLONNES A TRAITER  ['codecommune', 'AGE_pop_22', 'AGE_propf_22', 'AGE_popf014_22', 'AGE_popf1539_22', 'AGE_popf4059_22', 'AGE_popf60p_22', 'AGE_poph014_22', 'AGE_poph1539_22', 'AGE_poph4059_22', 'AGE_poph60p_22', 'AGE_prop014_22', 'AGE_prop1539_22', 'AGE_prop4059_22', 'AGE_prop60p_22', 'AGE_propf014_22', 'AGE_propf1539_22', 'AGE_propf4059_22', 'AGE_propf60p_22', 'AGE_age_22', 'AGE_agef_22', 'AGE_ageh_22', 'AGE_pop_12', 'AGE_propf_12', 'AGE_popf014_12', 'AGE_popf1539_12', 'AGE_popf4059_12', 'AGE_popf60p_12', 'AGE_poph014_12', 'AGE_poph1539_12', 'AGE_poph4059_12', 'AGE_poph60p_12', 'AGE_prop014_12', 'AGE_prop15

In [9]:
synthese

{'final_TAR_YYY_leg2022comm.csv': 15,
 'final_RSA_rsacommunes.csv': 4,
 'final_AGE_agesexcommunes.csv': 84,
 'final_BFL_basesfiscalescommunes.csv': 26,
 'final_PIB_pibcommunes-ratio-et-tot.csv': 8,
 'final_GEO_correspondance-code-insee-code-postal.csv': 2,
 'final_KP0_capitalimmobilier.csv': 10,
 'final_REV_revcommunes-revmoyfoy.csv': 4,
 'final_KP1_capitalimmobiliercommunes.csv': 8,
 'final_PO0_popcommuneselecteurs.csv': 4,
 'final_PRO_proprietairescommunes.csv': 8,
 'final_EMP_empfoncommunes.csv': 28,
 'final_DIP_diplomescommunes.csv': 20,
 'final_CED_crimesdelits.csv': 10,
 'final_CSP_cspcommunes.csv': 28,
 'final_TER_terrescommunes.csv': 4,
 'final_NAT_naticommunes.csv': 9}

In [10]:
df_final['key']=df_final['codecommune']

In [11]:
df_final.set_index('key', inplace=True)
df_final.head(2)

,dep,nomdep,codecommune,nomcommune,TAR_inscrits,TAR_votants,TAR_exprimes,YYY_pvoixNUP,YYY_pvoixECO,YYY_pvoixENS,...,TER_surf68,NAT_petranger22,NAT_pimmigre22,NAT_pimmnatur22,NAT_pnatur22,NAT_petranger12,NAT_pimmigre12,NAT_pimmnatur12,NAT_petranger02,NAT_petranger92
key,,,,,,,,,,,,,,,,,,,,,
01001,01,AIN,01001,L'ABERGEMENT-CLÉMENCIAT,644,343,339.0,0.153392,0.023599,0.147493,...,0.0,0.005161,0.014194,0.009032,0.014194,0.025740,0.037323,0.011583,0.018543,0.014634
01002,01,AIN,01002,L'ABERGEMENT-DE-VAREY,218,133,128.0,0.343750,0.000000,0.000000,...,0.0,0.003891,0.003891,0.000000,0.000000,0.012766,0.021277,0.008511,0.005618,0.037736


In [12]:
index_suppr = ['13201',
 '13202',
 '13203',
 '13204',
 '13205',
 '13206',
 '13207',
 '13208',
 '13209',
 '13210',
 '13211',
 '13212',
 '13213',
 '13214',
 '13215',
 '13216',
 '69380',
 '69381',
 '69382',
 '69383',
 '69384',
 '69385',
 '69386',
 '69387',
 '69388',
 '69389',
 '75056']



In [13]:
df_final = df_final[~df_final['codecommune'].isin(index_suppr)]
df_final.shape

(33803, 272)

In [14]:
rc=''
cpte = -1
for i,c in enumerate (df_final) :
    cpte+=1
    if c[0:3] != rc and i>3 :
        print('\n>>>> ',rc,':  + ',cpte, ' = ',i)
        rc = c[0:3]
        cpte = 0
    print (c,end=' - ')
print('\n>>>> ',rc,':  + ',cpte, ' = ',i)

dep - nomdep - codecommune - nomcommune - 
>>>>   :  +  4  =  4
TAR_inscrits - TAR_votants - TAR_exprimes - 
>>>>  TAR :  +  3  =  7
YYY_pvoixNUP - YYY_pvoixECO - YYY_pvoixENS - 
>>>>  YYY :  +  3  =  10
TAR_electeurs - 
>>>>  TAR :  +  1  =  11
YYY_pvoixLR_UDI - YYY_pvoixEXTREMD - YYY_pvoixCENTRE - YYY_pvoixOTR - 
>>>>  YYY :  +  4  =  15
RSA_nrsa2017 - RSA_nrsa2021 - RSA_prsa2017 - RSA_prsa2021 - 
>>>>  RSA :  +  4  =  19
AGE_pop_22 - AGE_propf_22 - AGE_popf014_22 - AGE_popf1539_22 - AGE_popf4059_22 - AGE_popf60p_22 - AGE_poph014_22 - AGE_poph1539_22 - AGE_poph4059_22 - AGE_poph60p_22 - AGE_prop014_22 - AGE_prop1539_22 - AGE_prop4059_22 - AGE_prop60p_22 - AGE_propf014_22 - AGE_propf1539_22 - AGE_propf4059_22 - AGE_propf60p_22 - AGE_age_22 - AGE_agef_22 - AGE_ageh_22 - AGE_pop_12 - AGE_propf_12 - AGE_popf014_12 - AGE_popf1539_12 - AGE_popf4059_12 - AGE_popf60p_12 - AGE_poph014_12 - AGE_poph1539_12 - AGE_poph4059_12 - AGE_poph60p_12 - AGE_prop014_12 - AGE_prop1539_12 - AGE_prop4059_12 

## TRAITER LES Nan RESTANTS

In [15]:
df_final.isna().sum().sum()

16465

In [16]:
nb_lines = len(df_final)
for c in df_final :
    nb_nan = df_final[c].isna().sum()
    if nb_nan > 0 :
        print (c,nb_nan, round(nb_nan/nb_lines*100,2),'%')

TAR_electeurs 47 0.14 %
RSA_nrsa2017 340 1.01 %
RSA_nrsa2021 340 1.01 %
RSA_prsa2017 340 1.01 %
RSA_prsa2021 340 1.01 %
AGE_propf_22 8 0.02 %
AGE_prop014_22 8 0.02 %
AGE_prop1539_22 8 0.02 %
AGE_prop4059_22 8 0.02 %
AGE_prop60p_22 8 0.02 %
AGE_propf014_22 1107 3.27 %
AGE_propf1539_22 726 2.15 %
AGE_propf4059_22 640 1.89 %
AGE_propf60p_22 341 1.01 %
AGE_age_22 8 0.02 %
AGE_pop_12 6 0.02 %
AGE_propf_12 6 0.02 %
AGE_popf014_12 6 0.02 %
AGE_popf1539_12 6 0.02 %
AGE_popf4059_12 6 0.02 %
AGE_popf60p_12 6 0.02 %
AGE_poph014_12 6 0.02 %
AGE_poph1539_12 6 0.02 %
AGE_poph4059_12 6 0.02 %
AGE_poph60p_12 6 0.02 %
AGE_prop014_12 6 0.02 %
AGE_prop1539_12 6 0.02 %
AGE_prop4059_12 6 0.02 %
AGE_prop60p_12 6 0.02 %
AGE_propf014_12 83 0.25 %
AGE_propf1539_12 16 0.05 %
AGE_propf4059_12 9 0.03 %
AGE_propf60p_12 8 0.02 %
AGE_age_12 9 0.03 %
AGE_agef_12 8 0.02 %
AGE_ageh_12 9 0.03 %
AGE_pop_02 27 0.08 %
AGE_propf_02 27 0.08 %
AGE_popf014_02 27 0.08 %
AGE_popf1539_02 27 0.08 %
AGE_popf4059_02 27 0.08 %
AGE_po

In [17]:
for c in df_final :
    print (c , type(c))

dep <class 'str'>
nomdep <class 'str'>
codecommune <class 'str'>
nomcommune <class 'str'>
TAR_inscrits <class 'str'>
TAR_votants <class 'str'>
TAR_exprimes <class 'str'>
YYY_pvoixNUP <class 'str'>
YYY_pvoixECO <class 'str'>
YYY_pvoixENS <class 'str'>
TAR_electeurs <class 'str'>
YYY_pvoixLR_UDI <class 'str'>
YYY_pvoixEXTREMD <class 'str'>
YYY_pvoixCENTRE <class 'str'>
YYY_pvoixOTR <class 'str'>
RSA_nrsa2017 <class 'str'>
RSA_nrsa2021 <class 'str'>
RSA_prsa2017 <class 'str'>
RSA_prsa2021 <class 'str'>
AGE_pop_22 <class 'str'>
AGE_propf_22 <class 'str'>
AGE_popf014_22 <class 'str'>
AGE_popf1539_22 <class 'str'>
AGE_popf4059_22 <class 'str'>
AGE_popf60p_22 <class 'str'>
AGE_poph014_22 <class 'str'>
AGE_poph1539_22 <class 'str'>
AGE_poph4059_22 <class 'str'>
AGE_poph60p_22 <class 'str'>
AGE_prop014_22 <class 'str'>
AGE_prop1539_22 <class 'str'>
AGE_prop4059_22 <class 'str'>
AGE_prop60p_22 <class 'str'>
AGE_propf014_22 <class 'str'>
AGE_propf1539_22 <class 'str'>
AGE_propf4059_22 <class 'str

In [18]:
infos = df_final.iloc[:, 0:4]
X_tmp= df_final.iloc[:, 4:]
infos.head


<bound method NDFrame.head of       dep      nomdep codecommune               nomcommune
key                                                       
01001  01         AIN       01001  L'ABERGEMENT-CLÉMENCIAT
01002  01         AIN       01002    L'ABERGEMENT-DE-VAREY
01004  01         AIN       01004        AMBÉRIEU-EN-BUGEY
01005  01         AIN       01005      AMBÉRIEUX-EN-DOMBES
01006  01         AIN       01006                  AMBLÉON
...    ..         ...         ...                      ...
95676  95  VAL-D'OISE       95676       VILLERS-EN-ARTHIES
95678  95  VAL-D'OISE       95678            VILLIERS-ADAM
95680  95  VAL-D'OISE       95680          VILLIERS-LE-BEL
95682  95  VAL-D'OISE       95682          VILLIERS-LE-SEC
95690  95  VAL-D'OISE       95690      WY-DIT-JOLI-VILLAGE

[33803 rows x 4 columns]>

In [19]:
# X_tmp.astype(int)
for c in X_tmp :
    print (c, end ='-')
    X_tmp[c] = pd.to_numeric(X_tmp[c], errors='coerce')

TAR_inscrits-TAR_votants-TAR_exprimes-YYY_pvoixNUP-YYY_pvoixECO-YYY_pvoixENS-TAR_electeurs-YYY_pvoixLR_UDI-YYY_pvoixEXTREMD-YYY_pvoixCENTRE-YYY_pvoixOTR-RSA_nrsa2017-RSA_nrsa2021-RSA_prsa2017-RSA_prsa2021-AGE_pop_22-AGE_propf_22-AGE_popf014_22-AGE_popf1539_22-AGE_popf4059_22-AGE_popf60p_22-AGE_poph014_22-AGE_poph1539_22-AGE_poph4059_22-AGE_poph60p_22-AGE_prop014_22-AGE_prop1539_22-AGE_prop4059_22-AGE_prop60p_22-AGE_propf014_22-AGE_propf1539_22-AGE_propf4059_22-AGE_propf60p_22-AGE_age_22-AGE_agef_22-AGE_ageh_22-AGE_pop_12-AGE_propf_12-AGE_popf014_12-AGE_popf1539_12-AGE_popf4059_12-AGE_popf60p_12-AGE_poph014_12-AGE_poph1539_12-AGE_poph4059_12-AGE_poph60p_12-AGE_prop014_12-AGE_prop1539_12-AGE_prop4059_12-AGE_prop60p_12-AGE_propf014_12-AGE_propf1539_12-AGE_propf4059_12-AGE_propf60p_12-AGE_age_12-AGE_agef_12-AGE_ageh_12-AGE_pop_02-AGE_propf_02-AGE_popf014_02-AGE_popf1539_02-AGE_popf4059_02-AGE_popf60p_02-AGE_poph014_02-AGE_poph1539_02-AGE_poph4059_02-AGE_poph60p_02-AGE_prop014_02-AGE_prop15

#### X_tmp.describe()

In [20]:
X_tmp.isna().sum()

TAR_inscrits       0
TAR_votants        0
TAR_exprimes       0
YYY_pvoixNUP       0
YYY_pvoixECO       0
                  ..
NAT_petranger12    0
NAT_pimmigre12     0
NAT_pimmnatur12    0
NAT_petranger02    0
NAT_petranger92    0
Length: 268, dtype: int64

In [21]:
from sklearn.impute import KNNImputer

In [22]:
KNNI = KNNImputer(n_neighbors=5)

In [23]:
from sklearn.impute import KNNImputer
imputer = KNNImputer(n_neighbors=5)
df_finalC = pd.DataFrame(imputer.fit_transform(X_tmp),columns=X_tmp.columns)

In [24]:
df_finalC.isna().sum().sum()

0

In [25]:
df_finalC.iloc[0:10].shape

(10, 268)

## EXPORTER

In [26]:
path_export='../DATA/final_DATA.csv'
df_finalC.to_csv(path_export, sep=';', index=True)
print(f'DataFrame exported to: {path_export}')

DataFrame exported to: ../DATA/final_DATA.csv


In [27]:
path_export='../DATA/final_4first_col.csv'
infos.to_csv(path_export, sep=';', index=True)
print(f'DataFrame exported to: {path_export}')

DataFrame exported to: ../DATA/final_4first_col.csv
